# Generische Datentypen

#### Patrick Schnider, Marcel Lüthi</br>Departement Mathematik und Informatik, Universität Basel

In der Programmierung möchte man oft Methoden so schreiben, dass sie für so viele Datentypen wie möglich funktionieren, und nur die wirklich nötige Struktur der Typen verwenden. Ein typisches Beispiel hierfür sind Sortieralgorithmen: solange wir beliebige zwei Elemente vergleichen können und so etwas wie "grösser" und "kleiner" haben, können wir sortieren. In Java ist diese Eigenschaft mit dem Interface ```Comparable<T>``` umgesetzt. Sie können sich das in der [API Dokumentation](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Comparable.html) ansehen.

In diesen Notebook wollen wir etwas ähnliches machen. Wir wollen mit Daten arbeiten, die einen Begriff von "ähnlich" haben.

### Das Interface

Wir definieren als erstes das relevante Interface.

In [ ]:
interface Similar<T> {
    boolean isSimilar(T other);
    String toString();
}

##### Fragen

* Welche Methoden würden Sie deklarieren?
* Wieso brauchen wir hier generische Datentypen?

### Eine erste Implementierung: Zahlen mit Fehlerbereich

Bei Messungen kriegt man oft nicht eindeutige Zahlen, sondern hat einen Fehlerbereich, zum Beispiel $73.2 +/- 0.8$. Wir können nun sagen, dass zwei Zahlen mit Fehlerbereich ähnlich sind, wenn sich die Fehlerbereiche überschneiden. In Java können wir das mit einer Klasse umsetzen, die unser Interface ```Similar<T>``` implementiert.

##### Übung

* Schreiben Sie die Klasse
* Wie müssen Sie die Klassensignatur schreiben?
* Wie testen Sie, ob sich zwei Fehlerbereiche überschneiden?

In [ ]:
class ApproxDouble implements Similar<ApproxDouble>{
    double value;
    double error;
    
    public ApproxDouble(double value, double error){
        this.value = value;
        this.error = error;
    }
    
    public String toString(){
        return "" + value + " +/- " + error;
    }
    
    public boolean isSimilar(ApproxDouble other){
        if (this.value + this.error < other.value - other.error){
            return false;
        }
        if (this.value - this.error > other.value + other.error){
            return false;
        }
        return true;
    }

}

Wir testen diese Klasse:

In [ ]:
class ApproxDoubleTest{

    public static void main(String[] args){
        ApproxDouble a = new ApproxDouble(0.0, 0.5);
        ApproxDouble b = new ApproxDouble(1.0, 0.6);
        ApproxDouble c = new ApproxDouble(1.0, 0.4);
        
        
        System.out.println(a.isSimilar(b)); // true
        System.out.println(a.isSimilar(c)); // false
        System.out.println(b.isSimilar(c)); // true
    }
    
}

ApproxDoubleTest.main(new String[0]);

### Ähnliche Paare ausgeben

Wir wollen nun eine Klasse schreiben, mit der wir ein Array von Objekten eines Datentyps, der ```Similar``` implementiert, speichern. Die Klasse soll eine Methode haben, die alle Paare von Objekten im Array ausgibt, die ähnlich sind.

##### Übung
* Schreiben Sie die Klasse
* Wie müssen Sie die Klassensignatur schreiben?

In [ ]:
class SimilarPairs<T extends Similar<T>>{
    T[] data;
    
    public SimilarPairs(T[] data){
        this.data = data;
    }
    
    public void printPairs(){
        for (int i = 0; i < data.length; i++){
            for (int j = 0; j < i; j++){
                T first = data[i];
                T second = data[j];
                if (first.isSimilar(second)){
                    System.out.println("(" + first.toString() + ", " + second.toString() + ")");
                }
            }
        }
    }
}

Dank unserer Implementierung ```ApproxDouble``` können wir diese Klasse an einem konkreten Beispiel testen.

In [ ]:
class SimilarPairsTest {

    public static void main(String[] args){
        ApproxDouble a = new ApproxDouble(0.0, 0.5);
        ApproxDouble b = new ApproxDouble(1.0, 0.6);
        ApproxDouble c = new ApproxDouble(1.0, 0.4);
        
        ApproxDouble[] doubleData = new ApproxDouble[]{a, b, c};
        SimilarPairs doubleTest = new SimilarPairs(doubleData);
        
        doubleTest.printPairs();
    }
    
}

SimilarPairsTest.main(new String[0]);

### Eine zweite Implementierung: Namen

Wir können Namen (oder allgemein Strings) vergleichen, indem wir zählen, wieviele gemeinsame Buchstaben sie haben. Zum Beispiel haben "Peter" und "Erich" 3 gemeinsame Buchstaben: 2 mal ein "e" und 1 mal ein "r". (Wir zählen das "e" doppelt, das könnte man natürlich auch anders machen. Mit unserer Definition haben "Peter" und "Esther" 4 gemeinsame "e".) Wir sagen nun, dass zwei Namen ähnlich sind, wenn sie mindestens 3 gemeinsame Buchtaben haben.

Auch hier können eine Klasse schreiben, die ```Similar<T>``` implementiert.

##### Übung
* Schreiben Sie die Klasse

In [ ]:
class Name implements Similar<Name> {
    String name;
    
    public Name(String name){
        this.name = name;
    }
    
    public String toString(){
        return this.name;
    }
    
    public boolean isSimilar(Name other){
        int commonChars = 0;
        for (int i = 0; i < this.name.length(); i = i + 1){
            char c = this.name.charAt(i);
            for (int j = 0; j < other.name.length(); j = j + 1){
                char d = other.name.charAt(j);
                if (c == d){commonChars++;}
            }
        }
        if (commonChars >= 3){return true;}
        else {return false;}
    }
}

Auch diese Klasse testen wir.

In [ ]:
class NameTest{

    public static void main(String[] args){
        Name patrick = new Name("patrick");
        Name peter = new Name("peter");
        Name erin = new Name("erin");
        
        System.out.println(patrick.isSimilar(peter)); // true
        System.out.println(patrick.isSimilar(erin)); // false
        System.out.println(peter.isSimilar(erin)); // true
    }
    
}

ApproxDoubleTest.main(new String[0]);

Wir können nun ```SimilarPairs``` auch mit ```Name``` anwenden.

In [ ]:
class SimilarPairsTest {

    public static void main(String[] args){
        // für ApproxDouble
        System.out.println("Paare mit ApproxDouble");
        
        ApproxDouble a = new ApproxDouble(0.0, 0.5);
        ApproxDouble b = new ApproxDouble(1.0, 0.6);
        ApproxDouble c = new ApproxDouble(1.0, 0.4);
        
        ApproxDouble[] doubleData = new ApproxDouble[]{a, b, c};
        SimilarPairs doubleTest = new SimilarPairs(doubleData);
        
        doubleTest.printPairs();
        
        // für Name
        System.out.println("Paare mit Name");
        
        Name patrick = new Name("patrick");
        Name peter = new Name("peter");
        Name erin = new Name("erin");
        
        Name[] nameData = new Name[]{patrick, peter, erin};
        SimilarPairs nameTest = new SimilarPairs(nameData);
        
        nameTest.printPairs();
    }
    
}

SimilarPairsTest.main(new String[0]);